In [5]:
%run script/setup.py

In [6]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
# print("Loading of data, time:", current_time)

# load data
df_invited = pd.read_csv('Data/factInvited.csv')
df_proceedings = pd.read_csv('Data/factProceedings.csv')

df = pd.concat([df_invited, df_proceedings], ignore_index=True)
pd.set_option('display.max_rows', 15)

full_name = df['Full name'].unique()

ids = []
orcids = []
works_count = []
results = []

BASE_URL = 'https://api.openalex.org/'
ENDPOINT = 'authors'
MAIL = 'elsa@itu.dk'

current_time = (datetime.now()).strftime("%H:%M:%S")
# print("Start author data retreival, time:", current_time)

for i in full_name[0:1]:

    filter = f'?search={i}' + ', from_publication_date:2019-01-01'
    mail = f'&mailto={MAIL}'
    complete_url = BASE_URL + ENDPOINT + filter + mail
    print(complete_url)

https://api.openalex.org/authors?search=Alex Sandy Pentland, from_publication_date:2019-01-01&mailto=elsa@itu.dk


# Citations information from OpenAlex

In [7]:
BASE_URL = 'https://api.openalex.org/'
ENDPOINT = 'authors'
MAIL = 'elsa@itu.dk'
mail = f'&mailto={MAIL}'

complete_url = BASE_URL + ENDPOINT

In [8]:
# load data
df_invited = pd.read_csv('Data/factInvited.csv')
df_proceedings = pd.read_csv('Data/factProceedings.csv')

df = pd.concat([df_invited, df_proceedings], ignore_index=True)
pd.set_option('display.max_rows', 15)

In [9]:
full_name = df['Full name'].unique()

len(full_name)

67381

Not taking care of exceptions

In [10]:
ids = []
orcids = []
works_count = []
results = []

for i in full_name[0:10]:

    print(i)

    filter = f'?search={i}'
    complete_url = BASE_URL + ENDPOINT + filter + mail
    response_json = requests.get(complete_url).json()
    author_data = response_json['results']

    results.append(len(author_data))

    # no results found
    if len(author_data) == 0:

        works_count.append([None])
        ids.append([None])
        orcids.append([None])

    # found only one results
    if len(author_data) == 1:

        works_count.append([author_data[0]['works_count']])
        ids.append([author_data[0]['id']])
        orcids.append([author_data[0]['orcid']])

    # found more results
    else:
        
        # store a list with the works count for every result
        works_count.append([author_data[result]['works_count'] for result in range(len(author_data))])
        # store a list with the ids for every result
        ids.append([author_data[result]['id'] for result in range(len(author_data))])
        # check on the orcid codes
        orcid_codes = [author_data[result]['orcid'] for result in range(len(author_data))]
        different_orcids = set(code for code in orcid_codes if code is not None)
        # manually adding a None value if no orcid code is found
        if not different_orcids:
            different_orcids.add(None)

        orcids.append(list(different_orcids))

Alex Sandy Pentland
Astro Teller
Daniel J. Clancy
Edward Feigenbaum
Ian Lane Davis
Martha E. Pollack
Paul Cohen
Peter Norvig
Russ B. Altman
Sebastian Thrun


Trying to take care of exceptions

In [12]:
chunk_size = 5

for i in range(0, len(full_name), chunk_size):

    chunk = full_name[i:i+chunk_size]
    print(chunk)

['Alex Sandy Pentland' 'Astro Teller' 'Daniel J. Clancy'
 'Edward Feigenbaum' 'Ian Lane Davis']
['Martha E. Pollack' 'Paul Cohen' 'Peter Norvig' 'Russ B. Altman'
 'Sebastian Thrun']
['Tim Finin' 'Amy Greenwald' 'Chitta Baral' 'Dana S. Nau'
 'James A. Hendler']
['Jay M. Tenenbaum' 'Jeff Hawkins' 'Marti Hearst' 'Martin A. Keane'
 'Marvin Minsky']
['Ronald J. Brachman' 'Sridhar Mahadevan' 'Tuckers Balch'
 'Bruce G. Buchanan' 'Dan Roth']
['Karen Myers' 'Ken Koedinger' 'Pedro M. Domingos' 'Tim Berners-Lee'
 'Alan C. Schultz']
['Alan Mackworth' 'Geoffrey S. F.' 'Lise Getoor' 'Matt Brown'
 'Michael Wooldridge']
['Oren Etzioni' 'Toby Walsh' 'Alexie A. Efros' 'Chris Urmson'
 'David Haussler']
['Eric Horvitz' 'Kenneth M. Ford' 'Lillian Lee' 'Mark Newman'
 'Seth Copen Goldstein']
['Stuart Russell' "Barry O'Sullivan" 'Carla P. Gomes' 'David C. Parkes'
 'Jey M. Tenenbaum']
['Leslie Pack Kaelbling' 'Majd Alwan' 'Michael Thielscher' 'Vernor Vinge'
 'David Ferrucci']
['David Gunning' 'Heinrich H. Buel

In [15]:
math.isnan(float('nan'))

True

In [38]:
[1]*10

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [48]:
failures = [True]
b = [False]*10
b[0] = True
b

[True, False, False, False, False, False, False, False, False, False]

In [49]:
chunk_size = 2

# creating an empty dataframe
columns = ['Full name', 'OpenAlex ID', 'ORCID', 'Works count']
df_author_id =  pd.DataFrame({col: [None]*len(full_name) for col in columns})

# DELETE THE LIMIT!!!
for j in range(0, len(full_name[0:10]), chunk_size):

    chunk = full_name[j:j+chunk_size]
    ids = []
    orcids = []
    works_count = []
    results = []
    failure = False

    for i in chunk:

        filter = f'?search={i}'
        complete_url = BASE_URL + ENDPOINT + filter + mail
        try:
            response = requests.get(complete_url)
            response.raise_for_status()
            response_json = response.json()
            author_data = response_json['results']

        except requests.exceptions.RequestException as e:
            print(f"Request failed for {i}: {e}")
            failure = True
            continue
        except ValueError as e:
            print(f"JSON decoding failed for {i}: {e}")
            failure = True
            continue

        results.append(len(author_data))

        # API failures
        if failure:

            # nan value is manually add to the arrays
            works_count.append([float('nan')])
            ids.append([float('nan')])
            orcids.append([float('nan')])

        # no results found
        elif len(author_data) == 0:

            works_count.append([None])
            ids.append([None])
            orcids.append([None])

        # found only one results
        elif len(author_data) == 1:

            works_count.append([author_data[0]['works_count']])
            ids.append([author_data[0]['id']])
            orcids.append([author_data[0]['orcid']])

        # found more results
        else:
            
            # store a list with the works count for every result
            works_count.append([author_data[result]['works_count'] for result in range(len(author_data))])
            # store a list with the ids for every result
            ids.append([author_data[result]['id'] for result in range(len(author_data))])
            # check on the orcid codes
            orcid_codes = [author_data[result]['orcid'] for result in range(len(author_data))]
            different_orcids = set(code for code in orcid_codes if code is not None)
            # manually adding a None value if no orcid code is found
            if not different_orcids:
                different_orcids.add(None)

            orcids.append(list(different_orcids))

    # the dataframe is filled after every chunk to not loose data
    df_author_id['Full name'][j:j+chunk_size] = chunk
    df_author_id['OpenAlex ID'][j:j+chunk_size] = ids
    df_author_id['ORCID'][j:j+chunk_size] = orcids
    df_author_id['Works count'][j:j+chunk_size] = works_count

True
True
0 2
[[nan], [nan]]
False
False
2 4
[['https://openalex.org/A5001852581'], ['https://openalex.org/A5052827274', 'https://openalex.org/A5006626552']]
False
False
4 6
[['https://openalex.org/A5043141178'], ['https://openalex.org/A5069313083', 'https://openalex.org/A5097912024']]
False
False
6 8
[['https://openalex.org/A5047214685', 'https://openalex.org/A5039490548', 'https://openalex.org/A5051700311', 'https://openalex.org/A5020060083', 'https://openalex.org/A5065586840', 'https://openalex.org/A5019586550', 'https://openalex.org/A5005297246', 'https://openalex.org/A5058812869', 'https://openalex.org/A5080646462', 'https://openalex.org/A5043308820', 'https://openalex.org/A5065810297', 'https://openalex.org/A5048380473', 'https://openalex.org/A5077853400', 'https://openalex.org/A5017087873', 'https://openalex.org/A5068502898', 'https://openalex.org/A5056803298', 'https://openalex.org/A5036338191', 'https://openalex.org/A5071569280', 'https://openalex.org/A5048116243', 'https://op

In [50]:
df_author_id

,Full name,OpenAlex ID,ORCID,Works count
0,Alex Sandy Pentland,[nan],[nan],[nan]
1,Astro Teller,[nan],[nan],[nan]
2,Daniel J. Clancy,[https://openalex.org/A5001852581],[None],[100]
3,Edward Feigenbaum,"[https://openalex.org/A5052827274, https://ope...",[None],"[199, 11]"
4,Ian Lane Davis,[https://openalex.org/A5043141178],[None],[15]
...,...,...,...,...
67376,None,None,None,None
67377,None,None,None,None
67378,None,None,None,None
67379,None,None,None,None


In [11]:
df_author_id = pd.DataFrame(columns = ['Full name', 'OpenAlex ID', 'ORCID', 'Works count'])
df_author_id['Full name'] = full_name[0:10]
df_author_id['OpenAlex ID'] = ids
df_author_id['ORCID'] = orcids
df_author_id['Works count'] = works_count

In [12]:
df_author_id

,Full name,OpenAlex ID,ORCID,Works count
0,Alex Sandy Pentland,"[https://openalex.org/A5067118505, https://ope...",[https://orcid.org/0000-0002-8053-9983],"[420, 148, 1, 1]"
1,Astro Teller,[https://openalex.org/A5046049741],[None],[20]
2,Daniel J. Clancy,[https://openalex.org/A5001852581],[None],[100]
3,Edward Feigenbaum,"[https://openalex.org/A5052827274, https://ope...",[None],"[199, 11]"
4,Ian Lane Davis,[https://openalex.org/A5043141178],[None],[15]
5,Martha E. Pollack,"[https://openalex.org/A5069313083, https://ope...",[None],"[218, 1]"
6,Paul Cohen,"[https://openalex.org/A5047214685, https://ope...","[https://orcid.org/0000-0001-5928-2127, https:...","[180, 543, 503, 225, 482, 189, 305, 74, 572, 1..."
7,Peter Norvig,"[https://openalex.org/A5080389385, https://ope...",[None],"[127, 2]"
8,Russ B. Altman,"[https://openalex.org/A5084043782, https://ope...",[https://orcid.org/0000-0003-3859-2905],"[611, 258, 1, 1, 0]"
9,Sebastian Thrun,"[https://openalex.org/A5066710774, https://ope...",[None],"[455, 1]"


In [14]:
filename = "authorID.csv" # or factProceedings.csv
filepath = "./Data/"
df_author_id.to_csv(os.path.join(filepath, filename), index=False)
print("The file is now saved")

The file is now saved


In [26]:
a = list(np.ones(10))
print(a)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


Better idea: create the empty dataframe and fill it chunck by chunck

In [27]:
columns = ['Full name', 'OpenAlex ID', 'ORCID', 'Works count']

# Create an empty DataFrame with 100 rows
df_empty = pd.DataFrame({col: [None]*len(full_name) for col in columns})
df_empty['Full name'][0 : 10] = a
df_empty['Full name'][67379] = 'si'
df_empty

,Full name,OpenAlex ID,ORCID,Works count
0,1.0,None,None,None
1,1.0,None,None,None
2,1.0,None,None,None
3,1.0,None,None,None
4,1.0,None,None,None
...,...,...,...,...
67376,None,None,None,None
67377,None,None,None,None
67378,None,None,None,None
67379,si,None,None,None


Figuring out how to build the for loop

In [1]:
# Sample list of 1000 elements
my_list = list(range(1000))

# Define chunk size
chunk_size = 100

# Iterate over the list in chunks of 100 elements
for i in range(0, len(my_list), chunk_size):
    chunk = my_list[i:i + chunk_size]
    print(chunk)  # Process the chunk as needed

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
[100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199]
[200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 22

Search for an author by its name

In [228]:
i = 'Stronzo Bestiale'
filter = f'?search={i}'
complete_url = BASE_URL + ENDPOINT + filter
response_json = requests.get(complete_url).json()
author_data = response_json['results']

In [229]:
author_data

[{'id': 'https://openalex.org/A5074178785',
  'orcid': None,
  'display_name': 'S. Bestiale',
  'display_name_alternatives': ['S. Bestiale', 'Stronzo Bestiale'],
  'relevance_score': 2494.2017,
  'works_count': 3,
  'cited_by_count': 189,
  'summary_stats': {'2yr_mean_citedness': 0.0, 'h_index': 2, 'i10_index': 2},
  'ids': {'openalex': 'https://openalex.org/A5074178785'},
  'affiliations': [{'institution': {'id': 'https://openalex.org/I129774422',
     'ror': 'https://ror.org/03prydq77',
     'display_name': 'University of Vienna',
     'country_code': 'AT',
     'type': 'education',
     'lineage': ['https://openalex.org/I129774422']},
    'years': [1987]}],
  'last_known_institutions': [{'id': 'https://openalex.org/I129774422',
    'ror': 'https://ror.org/03prydq77',
    'display_name': 'University of Vienna',
    'country_code': 'AT',
    'type': 'education',
    'lineage': ['https://openalex.org/I129774422']}],
  'topics': [{'id': 'https://openalex.org/T10591',
    'display_name':

Notes on how to retrieve the works of an autor from a start to an end date

In [ ]:
id = df_author_id['OpenAlex ID'][0][0]
id = id.split('/')[-1]
ENDPOINT =  'works'
filter = f'?filter=authorships.author.id:{id},from_publication_date:{start_date},to_publication_date:{end_date}'
complete_url = BASE_URL + ENDPOINT + filter + mail
print(complete_url)

Prova

In [ ]:
id = df_author_id['OpenAlex ID'][0][0]
id = id.split('/')[-1]
ENDPOINT =  'author'
filter = f'?filter=authorships.author.id:{id},from_publication_date:{start_date},to_publication_date:{end_date}'
complete_url = BASE_URL + ENDPOINT + filter + mail
print(complete_url)